In [1]:
#Load packages
import pandas as pd
import numpy as np

# I. Coding Run Expectancy Dataset (2017)

In [4]:
# Read in MLBAM Data for 2017

MLBAM17 = pd.read_csv("../Data/MLBAM17.csv")

In [5]:
# Keep only the relevant columns
RE17 = MLBAM17[['batterName','batterId','event', 'start1B', 'start2B', 'start3B', 'end1B', 'end2B', 'end3B',\
                   'startOuts','endOuts','runsFuture','runsOnPlay','outsInInning',\
                   'stand', 'throws','venueId', 'stadium', 'batterPos']].copy()

RE17.head()        

,batterName,batterId,event,start1B,start2B,start3B,end1B,end2B,end3B,startOuts,endOuts,runsFuture,runsOnPlay,outsInInning,stand,throws,venueId,stadium,batterPos
0,Gardner,458731,Flyout,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,3,L,R,12,Tropicana Field,LF
1,"Sanchez, G",596142,Groundout,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0,0,3,R,R,12,Tropicana Field,C
2,Bird,595885,Walk,NaN,NaN,NaN,595885.0,NaN,NaN,2,2,0,0,3,L,R,12,Tropicana Field,1B
3,Holliday,407812,Groundout,595885.0,NaN,NaN,NaN,NaN,NaN,2,3,0,0,3,R,R,12,Tropicana Field,DH
4,"Dickerson, C",572816,Single,NaN,NaN,NaN,572816.0,NaN,NaN,0,0,3,0,3,L,R,12,Tropicana Field,DH


In [6]:
# Create three indicator variables (one for each base) to denote whether or not each base is occupied prior to the plate appearance
RE17['Start1'] = np.where(pd.isnull(RE17['start1B']),0,1)
RE17['Start2'] = np.where(pd.isnull(RE17['start2B']),0,1)
RE17['Start3'] = np.where(pd.isnull(RE17['start3B']),0,1)

In [7]:
# Create a “start state” variable to denote the base out state prior to each plate appearance
RE17['Start_State'] = (RE17['Start1'].astype(str) + RE17['Start2'].astype(str) + RE17['Start3'].astype(str)+\
                          " " + RE17['startOuts'].astype(str))
RE17['Start_State'].head()

0    000 0
1    000 1
2    000 2
3    100 2
4    000 0
Name: Start_State, dtype: object

In [8]:
# Create three indicator variables (one for each base) to denote whether or not each base is occupied after the plate appearance
RE17['End1'] = np.where(pd.isnull(RE17['end1B']),0,1)
RE17['End2'] = np.where(pd.isnull(RE17['end2B']),0,1)
RE17['End3'] = np.where(pd.isnull(RE17['end3B']),0,1)

In [9]:
# Create an “end state” variable to denote the base out state after each plate appearance 
RE17['End_State'] = (RE17['End1'].astype(str) + RE17['End2'].astype(str) + RE17['End3'].astype(str) + \
                        " " + RE17['endOuts'].astype(str))

In [10]:
# Restrict data to plays where either there is a change in state/runs scored
# and innings in which there were exactly 3 outs recorded
RE17 = RE17[((RE17["Start_State"] != RE17["End_State"]) | (RE17["runsOnPlay"] > 0)) & (RE17["outsInInning"] == 3)]

The MLBAM data conveniently includes the value of the runs scored from the beginning of each event to the end of the team's inning - this is the variable 'runsFuture'. So now we can simply use .groupby to calculate the average number of runs scored in the 24 starting states we have coded.

In [14]:
# Calculate run expectancy by starting state and merge this into play by play data
Start_RunExp = RE17.groupby(['Start_State'])['runsFuture'].mean().reset_index().rename(columns={'runsFuture':'Start_RE'})
Start_RunExp.head()
RE17 = pd.merge(RE17, Start_RunExp, on=['Start_State'], how='left')

,Start_State,Start_RE
0,000 0,0.516375
1,000 1,0.272176
2,000 2,0.108038
3,001 0,1.436482
4,001 1,0.953586


A team's inning ends when there are three outs, but can never start with three outs. To calculate the run value of an event we need to include the run expectancy of these states, which is by definition is zero, since no more runs can be scored when the inning is ended. Thus we can define End_RunExp as being equal to Start_RunExp, only with the eight extra three outs states, each with zero run expectancy. So now we create those states and append them to the Start_RunExp df and rename it as End_RunExp.

In [15]:
# Using the run expectancy data by starting state, create a variable to denote run expectancy by ending state
#(in addition, make sure to include the base out states
# for which there are 3 outs with run expectancies of 0 to the run expectancy by ending state data) 

Base_State_3 = [pd.Series(['000 3', 0], index=Start_RunExp.columns),
                pd.Series(['001 3', 0], index=Start_RunExp.columns),
                pd.Series(['010 3', 0], index=Start_RunExp.columns),
                pd.Series(['011 3', 0], index=Start_RunExp.columns),
                pd.Series(['100 3', 0], index=Start_RunExp.columns),
                pd.Series(['101 3', 0], index=Start_RunExp.columns),
                pd.Series(['110 3', 0], index=Start_RunExp.columns),
                pd.Series(['111 3', 0], index=Start_RunExp.columns)]

Base_State_3

[Start_State    000 3
 Start_RE           0
 dtype: object, Start_State    001 3
 Start_RE           0
 dtype: object, Start_State    010 3
 Start_RE           0
 dtype: object, Start_State    011 3
 Start_RE           0
 dtype: object, Start_State    100 3
 Start_RE           0
 dtype: object, Start_State    101 3
 Start_RE           0
 dtype: object, Start_State    110 3
 Start_RE           0
 dtype: object, Start_State    111 3
 Start_RE           0
 dtype: object]

In [17]:
Start_RunExp = Start_RunExp.append(Base_State_3, ignore_index=True)
Start_RunExp.tail(10)

,Start_State,Start_RE
22,111 1,1.635608
23,111 2,0.804609
24,000 3,0.000000
25,001 3,0.000000
26,010 3,0.000000
27,011 3,0.000000
28,100 3,0.000000
29,101 3,0.000000
30,110 3,0.000000
31,111 3,0.000000


In [19]:
End_RunExp = Start_RunExp.rename(columns={'Start_State':'End_State', 'Start_RE':'End_RE'})

In [20]:
# Merge the ending state run expectancy into the play by play data
RE17 = pd.merge(RE17, End_RunExp, on=['End_State'], how='left')

In [21]:
# Calculate the run value of each event
RE17['Run_Value'] = RE17['runsOnPlay'] + RE17['End_RE'] - RE17['Start_RE']

Question 1
What percent of plate appearances resulted in fly outs in 2017?

In [22]:
RE17['event'].unique()

array(['Flyout', 'Groundout', 'Walk', 'Single', 'Double', 'Sac Fly',
       'Lineout', 'Strikeout', 'Home Run', 'Pop Out', 'Forceout',
       'Hit By Pitch', 'Field Error', 'Triple', 'Grounded Into DP',
       'Double Play', 'Runner Out', 'Sac Bunt', 'Strikeout - DP',
       'Bunt Groundout', 'Intent Walk', 'Sac Fly DP',
       'Catcher Interference', 'Fielders Choice Out', 'Bunt Pop Out',
       'Fielders Choice', 'Fan interference', 'Batter Interference',
       'Bunt Lineout', 'Triple Play'], dtype=object)

In [24]:
len(RE17[ RE17['event'] == 'Flyout' ])/ len(RE17)

0.10697100421648598

Question 2
How many plate appearances had a starting base state in which the bases were loaded (all bases were occupied)?

In [26]:
len(RE17[(RE17['Start1'] ==1) & (RE17['Start2'] ==1) & (RE17['Start3'] == 1)])

4364

Question 3
Calculate aggregate strikeouts by player position (that is, aggregate (sum) data at the positional level and not the player level).  What was the highest aggregate strikeout total by position?

In [30]:
RE17["S/O"] = np.where( RE17["event"] == "Strikeout", 1, 0)
Batter_Pos_SO = RE17.groupby("batterPos")["S/O"].sum().reset_index()
Batter_Pos_SO[ Batter_Pos_SO["S/O"] == Batter_Pos_SO["S/O"].max() ]

,batterPos,S/O
0,1B,4399


# II. Coding Run Expectancy Dataset (2016)

In [31]:
# Read in MLBAM Data for 2016
MLBAM16 = pd.read_csv("../Data/MLBAM16.csv")

In [32]:
# Keep only the relevant columns (see exercise notebook)
RE16 = MLBAM16[['batterName','batterId','event', 'start1B', 'start2B', 'start3B', 'end1B', 'end2B', 'end3B',\
                   'startOuts','endOuts','runsFuture','runsOnPlay','outsInInning',\
                   'stand', 'throws','venueId', 'stadium', 'batterPos']].copy()

In [33]:
# Create three indicator variables (one for each base) to denote
# whether or not each base is occupied prior to the plate appearance
RE16['Start1'] = np.where(pd.isnull(RE16['start1B']),0,1)
RE16['Start2'] = np.where(pd.isnull(RE16['start2B']),0,1)
RE16['Start3'] = np.where(pd.isnull(RE16['start3B']),0,1)

In [34]:
# Create a “start state” variable to denote the base out state prior to each plate appearance
RE16['Start_State'] = (RE16['Start1'].astype(str) + RE16['Start2'].astype(str) + RE16['Start3'].astype(str)+\
                          " " + RE16['startOuts'].astype(str))
RE16['Start_State'].head()

0    000 0
1    000 1
2    000 2
3    000 0
4    000 1
Name: Start_State, dtype: object

In [35]:
# Create three indicator variables (one for each base) to denote whether or not each base is occupied after the plate appearance
RE16['End1'] = np.where(pd.isnull(RE16['end1B']),0,1)
RE16['End2'] = np.where(pd.isnull(RE16['end2B']),0,1)
RE16['End3'] = np.where(pd.isnull(RE16['end3B']),0,1)

In [36]:
# Create an “end state” variable to denote the base out state after each plate appearance
RE16['End_State'] = (RE16['End1'].astype(str) + RE16['End2'].astype(str) + RE16['End3'].astype(str) + \
                        " " + RE16['endOuts'].astype(str))

In [37]:
# Restrict data to plays where either there is a change in state/runs scored and innings in which there were exactly 3 outs recorded
RE16 = RE16[((RE16["Start_State"] != RE16["End_State"]) | (RE16["runsOnPlay"] > 0)) & (RE16["outsInInning"] == 3)]

In [38]:
# Calculate run expectancy by starting state and merge this into play by play data
Start_RunExp = RE16.groupby(['Start_State'])['runsFuture'].mean().reset_index().rename(columns={'runsFuture':'Start_RE'})
Start_RunExp.head()
RE16 = pd.merge(RE16, Start_RunExp, on=['Start_State'], how='left')

In [39]:
# Using the run expectancy data by starting state, create a variable to denote run expectancy by ending state (in addition, make sure to include the base out states for which there are 3 outs with run expectancies of 0 to the run expectancy by ending state data) 

Base_State_3 = [pd.Series(['000 3', 0], index=Start_RunExp.columns),
                pd.Series(['001 3', 0], index=Start_RunExp.columns),
                pd.Series(['010 3', 0], index=Start_RunExp.columns),
                pd.Series(['011 3', 0], index=Start_RunExp.columns),
                pd.Series(['100 3', 0], index=Start_RunExp.columns),
                pd.Series(['101 3', 0], index=Start_RunExp.columns),
                pd.Series(['110 3', 0], index=Start_RunExp.columns),
                pd.Series(['111 3', 0], index=Start_RunExp.columns)]


In [40]:
Start_RunExp = Start_RunExp.append(Base_State_3, ignore_index=True)
Start_RunExp.tail(10)

,Start_State,Start_RE
22,111 1,1.536937
23,111 2,0.696658
24,000 3,0.000000
25,001 3,0.000000
26,010 3,0.000000
27,011 3,0.000000
28,100 3,0.000000
29,101 3,0.000000
30,110 3,0.000000
31,111 3,0.000000


In [41]:
End_RunExp = Start_RunExp.rename(columns={'Start_State':'End_State', 'Start_RE':'End_RE'})

In [42]:
# Merge the ending state run expectancy into the play by play data
RE16 = pd.merge(RE16, End_RunExp, on=['End_State'], how='left')

In [43]:
# Calculate the run value of each event
RE16['Run_Value'] = RE16['runsOnPlay'] + RE16['End_RE'] - RE16['Start_RE']

Question 1
What percent of plate appearances resulted in ground outs in 2016?

In [44]:
len(RE16[ RE16['event'] == 'Groundout' ])/ len(RE16)

0.18445088839654197

Question 2
How many plate appearances had a starting base state in which at least one base was occupied?

In [45]:
len(RE16[(RE16['Start1'] ==1) | (RE16['Start2'] ==1) | (RE16['Start3'] == 1)])

80387

Question 3
Calculate aggregate home runs by player position (that is, aggregate (sum) data at the positional level and not the player level).  What was the highest aggregate home run total by position?

In [46]:
RE16["HR"] = np.where( RE16["event"] == "Home Run", 1, 0)
Batter_Pos_SO = RE16.groupby("batterPos")["HR"].sum().reset_index()
Batter_Pos_SO[ Batter_Pos_SO["HR"] == Batter_Pos_SO["HR"].max() ]

,batterPos,HR
0,1B,797


# III. Comparing 2016 vs. 2017

In [48]:
# Calculate the average run value for each event for both 2016 and 2017.
run_value_event_16 = RE16.groupby("event")['Run_Value'].mean().reset_index()
run_value_event_17 = RE17.groupby("event")['Run_Value'].mean().reset_index()

In [49]:
# Merge the 2016 and 2017 run values by event into one dataframe
# (before merging, it may be helpful to rename the run value variables Run_Value16 and Run_Value17 respectively)
run_value_event_16 = run_value_event_16.rename(columns={'Run_Value':'RV_16'})
run_value_event_17 = run_value_event_17.rename(columns={'Run_Value':'RV_17'})
run_value_event = pd.merge( run_value_event_16, run_value_event_17, on = "event")

In [60]:
# Calculate the aggregate run value (sum) for all players in both 2016 and 2017
run_value_player_16 = RE16.groupby("batterId")['Run_Value'].sum().reset_index()
run_value_player_17 = RE17.groupby("batterId")['Run_Value'].sum().reset_index()

In [61]:
# Merge the 2016 and 2017 run values by player into one dataframe
# so that only players appearing in both datasets appear in the merged data
# (before merging, it may be helpful to rename the run value variables Run_Value16 and Run_Value17 respectively)
run_value_player_16 = run_value_player_16.rename(columns={'Run_Value':'RV_16'})
run_value_player_17 = run_value_player_17.rename(columns={'Run_Value':'RV_17'})
run_value_player = pd.merge( run_value_player_16, run_value_player_17, on = "batterId", how = "outer")

Question 1
What was the highest player run value in 2017?

In [62]:
run_value_player_17[ run_value_player_17["RV_17"] == run_value_player_17["RV_17"].max()]

,batterId,RV_17
169,458015,67.650742


Question 2
What was the lowest player run value in 2016?

In [63]:
run_value_player_16[ run_value_player_16["RV_16"] == run_value_player_16["RV_16"].min()]

,batterId,RV_16
698,588751,-32.142529


Question 3
For each event, calculate the difference in run value between 2017 and 2016 (RV 2017-RV 2016).  Which event saw the largest change (in absolute value) from 2016 to 2017?

In [64]:
run_value_event["RV_diff"] = abs(run_value_event["RV_17"] - run_value_event["RV_16"])

In [65]:
run_value_event[run_value_event["RV_diff"] == run_value_event["RV_diff"].max()]

,event,RV_16,RV_17,RV_diff
23,Sac Fly DP,-0.370443,-0.543715,0.173272


Question 4
Calculate the difference in player run value between 2017 and 2016 for players that accumulated run values in both seasons.  According to this calculation, what was the largest improvement in run value from 2016 to 2017?

In [66]:
run_value_player["RV_diff"] = run_value_player["RV_17"] - run_value_player["RV_16"]

In [67]:
run_value_player[run_value_player["RV_diff"] == run_value_player["RV_diff"].max()]

,batterId,RV_16,RV_17,RV_diff
722,592450,-4.179157,57.081625,61.260782
